# Advent of Code 2024

> If debugging is the process of removing software bugs, then programming must be the process of putting them in.

-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
import math
import re
from itertools import chain, islice, product, count, takewhile, batched, repeat, accumulate
from functools import cache, reduce
from urllib import request
from heapq import heappush, heappop
from collections import Counter, defaultdict
from math import prod, inf



def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2024/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

Many problems this year require a _grid_, a 2d array that usually contains characters at each coordinates.

In [2]:
class Grid:
    def __init__(self, raw):
        self.rows = [*map(list, raw.split('\n'))]
        self.height = len(self.rows)
        self.width = len(self.rows[0])

    def __getitem__(self, u):
        if u in self: return self.rows[u[0]][u[1]]
        else: raise IndexError()

    def __setitem__(self, u, n):
        self.rows[u[0]][u[1]] = n
        
    __contains__ = lambda s, u: (0 <= u[0] < s.height) and (0 <= u[1] < s.width)

## [Day 1: Historian Hysteria](https://adventofcode.com/2024/day/1)

In [3]:
a, b = [list(t) for t in zip(*[[*map(int, x.split())] for x in aocin(1).split('\n')])]
na, nb = Counter(a), Counter(b)

A = sum(abs(u-v) for u, v in zip(sorted(a), sorted(b)))
assert A == 1388114

A = sum(u * nu * nb[u] for u, nu in na.items())
assert A == 23529853

## [Day 2: Red-Nosed Reports](https://adventofcode.com/2024/day/2)

In [4]:
reports = [[*map(int, l.split())] for l in aocin(2).split('\n')]

def is_safe(l, allowed, tol=0):
    def _f(l, tol):
        if tol < 0: return False
        if len(l) < 2: return True
        a, b, *rest = l
        return (
            ((a is None or a-b in allowed) and _f([b] + rest, tol))
            or _f([a] + rest, tol-1)
        )
    return _f([None] + l, tol)


A = sum(is_safe(l, {1, 2, 3}) or is_safe(l, {-1, -2, -3}) for l in reports)
assert A == 369

A = sum(is_safe(l, {1, 2, 3}, tol=1) or is_safe(l, {-1, -2, -3}, tol=1) for l in reports)
assert A == 428

## [Day 3: Mull it Over](https://adventofcode.com/2024/day/3)

In [5]:
A = sum(
    prod(map(int, m.groups()))
    for m in re.finditer(r'mul\((\d{1,3}),(\d{1,3})\)', aocin(3))
)
assert A == 160672468


def mul_with_switch(input):
    active, acc = True, 0
    for m in re.finditer(r"(do|don't|mul)\((\d{0,3}),?(\d{0,3})\)", input):
        match m.groups():
            case "mul", _ as a, _ as b:
                acc += active * int(a) * int(b)
            case "do", _, _:
                active = True
            case "don't", _, _:
                active = False
    return acc

A = mul_with_switch(aocin(3))
assert A == 84893551

## [Day 4: Ceres Search](https://adventofcode.com/2024/day/4)

In [6]:
def gen_input(raw):
    rows = raw.split('\n')
    cols = [''.join(u) for u in list(zip(*rows))]
    diag_primary = defaultdict(str)
    diag_secondary = defaultdict(str)
    for i in range(len(rows)):
        for j in range(len(rows[0])):
            diag_primary[i-j] += rows[i][j]
            diag_secondary[i+j] += rows[i][j]
    return rows, cols, diag_primary.values(), diag_secondary.values()


rows, cols, diag_primary, diag_secondary = gen_input(aocin(4))
A = sum(
    u.count('XMAS') + u.count('SAMX')
    for u in chain(rows, cols, diag_primary, diag_secondary)
)
assert A == 2613

A = sum(
    (
            rows[i][j] == 'A'
        and {rows[i+1][j+1], rows[i-1][j-1]} == {'S', 'M'}
        and {rows[i+1][j-1], rows[i-1][j+1]} == {'S', 'M'} 
    )
    for i in range(1, len(rows) - 1)
    for j in range(1, len(rows[0]) - 1)
)
assert A == 1905

## [Day 5: Print Queue](https://adventofcode.com/2024/day/5)

The algorithms and data structures begin to strike! :)

It's easy to observe that the problem is asking questions related to the topological ordering of the graph of rules.

The second part of the problem is actually underspecified. The text of the problem seems to be assuming that there exists a unique topological order, but that is obviously not the case, for a trivial counterexample imagine an empty rule graph and a nonempty update array; any permutation of the array would be valid.

In [7]:
rules, updates = aocin(5).split('\n\n')
updates_lst = [[*map(int, r.split(','))] for r in updates.split('\n')]

def rules_graphs(rules):
    direct, reverse = defaultdict(set), defaultdict(set)
    for rule in rules.split('\n'):
        a, b = map(int, rule.split('|'))
        direct[b].add(a), reverse[a].add(b)
    return direct, reverse

def is_inorder(rules_graph, update):
    present, done = set(update), set()
    for u in update:
        if any(v in present and v not in done for v in rules_graph[u]): return False
        done.add(u)
    return True

def topological_sort(direct, reverse, update):
    present = set(update)
    out_degree = {k: sum(u in present for u in direct[k]) for k in present}
    active = set(k for k, v in out_degree.items() if v == 0) 
    while active:
        yield (x := active.pop())
        out_degree[x] = -1
        for v in reverse[x]:
            if v in out_degree: out_degree[v] -= 1
            if out_degree.get(v, -1) == 0: active.add(v)


direct, reverse = rules_graphs(rules)

A = sum(is_inorder(direct, u) * u[len(u) // 2] for u in updates_lst)
assert A == 4185

A = sum(
    next(islice(topological_sort(direct, reverse, u), len(u) // 2, len(u) // 2 + 1))
    for u in updates_lst if not is_inorder(direct, u)
)
assert A == 4480

## [Day 6 - Guard Gallivant](https://adventofcode.com/2024/day/6)

In [8]:
class GuardGrid(Grid):
    def __init__(self, raw):
        super().__init__(raw)
        for i in range(self.height):
            for j in range(self.width):
                if self[i, j] == '^':
                    self.starting_point = (i, j)
                    break


def get_visited(grid):
    i, j = grid.starting_point
    di, dj = -1, 0
    yield i, j
    while True:
        ni, nj = i + di, j + dj
        try:
            match grid[ni, nj]:
                case '#': di, dj = dj, -di
                case  _ : i, j = i+di, j+dj; yield i, j
        except IndexError: 
            break

@cache
def compute_next(grid, i, j, di, dj):
    pi, pj = i, j
    try:
        while grid[pi, pj] != '#':
            pi, pj = pi+di, pj+dj
        return pi-di, pj-dj
    except IndexError:
        return None, None

def generate_loops(grid):
    for h, k in set(get_visited(grid)):
        seen = set()
        i, j = grid.starting_point
        di, dj = -1, 0
        while True:
            if (i, j) == (None, None): break
            if (i, j, di, dj) in seen: yield h, k; break
            else: seen.add((i, j, di, dj))
            ni, nj = compute_next(grid, i, j, di, dj)
            if   (di, dj) == (0, 1)  and i == h and j <= k <= (nj or inf) : i, j = i, k-1
            elif (di, dj) == (0, -1) and i == h and (nj or -inf) <= k <= j: i, j = i, k+1
            elif (di, dj) == (1, 0)  and j == k and i <= h <= (ni or inf) : i, j = h-1, j
            elif (di, dj) == (-1, 0) and j == k and (ni or -inf) <= h < i : i, j = h+1, j
            else: i, j = ni, nj
            di, dj = dj, -di


A = len(set(get_visited(GuardGrid(aocin(6)))))
assert A == 4939

A = sum(1 for _ in generate_loops(GuardGrid(aocin(6))))
assert A == 1434

## [Day 7: Bridge Repair](https://adventofcode.com/2024/day/7)

The interesting idea is that doing operations in reverse massively cuts down the state space.

Let $f(n, L)$ be a binary function that is true if the list $L$ can possibly reach the target $n$. Because the operations are always performed left to right, the choice of operations doesn't change the order, and we may look at the last element of the list:

- If $L$ has only one element, then $f(n, L)$ is true if and only if that element is $n$.
- Otherwise, let $k$ be the last element of $L$. Then, $f(n, L' \circ k) = f(n-k, L') \vee f(n/k, L')$.

Because no $n$ is reachable if $n$ is not a nonnegative integer, a lot of recursive calls may be immediately discarded, unlike in the more naive algorithm.
Introducing the catenation operation is not problematic, operations are still done left to right and the same logic applies with minor variations.


In [9]:
def gen_lines(raw):
    lines = []
    for line in raw.split('\n'):
        target, ls = line.split(':')
        lines.append((int(target), [*map(int, ls.split())]))
    return lines


def can_result_in(target, elem, with_concat=False):
    *l, last = elem
    if not l: return last == target
    return (
        (
            target >= last
            and can_result_in(target - last, l, with_concat)
        ) or (
            (target % last == 0)
            and can_result_in(target // last, l, with_concat)
        ) or (
            with_concat 
            and (str(target).endswith(str(last)))
            and can_result_in(int(str(target)[:-len(str(last))] or '0'), l, with_concat)
        )
    )

   
A = sum(k for k, v in gen_lines(aocin(7)) if can_result_in(k, v))
assert A == 5030892084481

A = sum(k for k, v in gen_lines(aocin(7)) if can_result_in(k, v, True))
assert A == 91377448644679

## [Day 8: Resonant Collinearity](https://adventofcode.com/2024/day/8)

In [10]:
class AntennaGrid(Grid):
    def __init__(self, raw):
        super().__init__(raw)
        self.antennas = defaultdict(list)
        for h in range(self.height):
            for k in range(self.width):
                if (u := self[h, k]) != '.':
                    self.antennas[u].append((h, k))


def get_antinodes(grid, min_drift=1, max_drift=1):
    for l in grid.antennas.values():
        for u, v in product(l, l):
            if u == v: continue
            dx, dy = v[0] - u[0], v[1] - u[1]
            for k in takewhile(lambda x: x <= max_drift, count(min_drift)):
                if (t := (u[0] - dx*k, u[1] - dy*k)) in grid: yield t
                else: break


A = len(set(get_antinodes(AntennaGrid(aocin(8)))))
assert A == 327

A = len(set(get_antinodes(AntennaGrid(aocin(8)), 0, inf)))
assert A == 1233

## [Day 9: Disk Fragmenter](https://adventofcode.com/2024/day/9)

In [11]:
def get_reordered(disk_map):
    blocks, free = [], []
    for n, t in enumerate(batched(disk_map, 2)):
        blocks.append((n, t[0]))
        free.append(t[1] if len(t) > 1 else 0)

    forward = chain(*(repeat(*l) for l in blocks))
    back = chain(*(repeat(*l) for l in reversed(blocks)))

    left = sum(x for _, x in blocks)
    for (_, n), m in zip(blocks, free):
        for _ in range(n):
            yield next(forward)
            if not (left := left - 1): return
        for _ in range(m):
            yield next(back)
            if not (left := left - 1): return

def get_reordered_contiguous(disk_map):
    blocks = []
    for n, t in enumerate(batched(disk_map, 2)):
        blocks.append((n, t[0]))
        blocks.append((None, t[1] if len(t) > 1 else 0))

    for e, n in reversed(list((a, b) for a, b in blocks if a is not None)):
        for i in range(len(blocks)):
            ee, nn = blocks[i]
            if ee == e: break
            elif ee is not None: continue
            elif nn >= n:
                ii = blocks.index((e, n))
                blocks[ii] = None, blocks[ii][1]
                blocks[i] = (ee, nn-n)
                if blocks[i][1] == 0: blocks.pop(i)
                blocks.insert(i, (e, n))
                break

    yield from chain(*(repeat(e or 0, n) for e, n in blocks))

disk_map = [*map(int, aocin(9))]
A = sum(n * e for n, e in enumerate(list(get_reordered(disk_map))))
assert A == 6310675819476

A = sum(n * e for n, e in enumerate(list(get_reordered_contiguous(disk_map))))
assert A == 6335972980679

## [Day 10: Hoof It](https://adventofcode.com/2024/day/10)

In [12]:
class TopoGrid(Grid):
    def __init__(self, raw):
        super().__init__(raw)
        self.rows = list(map(lambda u: list(map(int, u)), self.rows))
        self.starting_points = {
            (i, j) for i, j in product(range(self.height), range(self.width))
            if self[i, j] == 9
        }

def get_trailheads(grid):
    trailheads = defaultdict(Counter)
    state = defaultdict(Counter, {k: Counter({k}) for k in grid.starting_points})
    while state:
        (i, j), starting = state.popitem()
        for tt in {(i+1, j), (i-1, j), (i, j+1), (i, j-1)}:
            try:
                if grid[*tt] == grid[i, j] - 1:
                    if grid[*tt] == 0: trailheads[tt] += starting
                    else: state[tt] |= starting
            except IndexError: continue
    return trailheads


A = sum(map(len, get_trailheads(TopoGrid(aocin(10))).values()))
assert A == 531

A = sum(map(lambda u: sum(u.values()), get_trailheads(TopoGrid(aocin(10))).values()))
assert A == 1210

## [Day 11: Plutonian Pebbles](https://adventofcode.com/2024/day/11)

In [13]:
@cache
def generated_stones(n, steps):
    if steps == 0: return 1
    if n == 0: return generated_stones(1, steps-1)
    if (u := len(str(n))) % 2 == 0:
        return (
              generated_stones(int(str(n)[:u//2]), steps-1)
            + generated_stones(int(str(n)[u//2:]), steps-1)
        )
    else: return generated_stones(2024*n, steps-1)


stones = list(map(int, aocin(11).split()))
A = sum(generated_stones(x, 25) for x in stones)
assert A == 218956

A = sum(generated_stones(x, 75) for x in stones)
assert A == 259593838049805

## [Day 12: Garden Groups](https://adventofcode.com/2024/day/12)

In [14]:
def get_garden_areas(grid):
    unseen = set(product(range(grid.height), range(grid.width)))
    while unseen:
        i, j = unseen.pop()
        area, perimeter, value, S = 1, set(), grid[i, j], [(i, j)]
        while S:
            x, y = S.pop()
            for xx, yy in {(x+1, y), (x-1, y), (x, y+1), (x, y-1)}:
                if (xx, yy) not in grid or grid[xx, yy] != value:
                    perimeter.add((xx, yy, xx-x, yy-y))
                elif grid[xx, yy] == value and (xx, yy) in unseen:
                    unseen.discard((xx, yy))
                    S.append((xx, yy))
                    area += 1
        sides, P = 0, set(perimeter)
        while P:
            S = [P.pop()]
            sides += 1
            while S:
                x, y, dx, dy = S.pop()
                for xx, yy, in {(x+1, y), (x-1, y), (x, y+1), (x, y-1)}:
                    if (t := (xx, yy, dx, dy)) in P:
                        P.discard(t)
                        S.append(t)
        yield area, len(perimeter), sides


A = sum(a * p for a, p, _ in get_garden_areas(Grid(aocin(12))))
assert A == 1471452

A = sum(a * s for a, _, s in get_garden_areas(Grid(aocin(12))))
assert A == 863366

## [Day 13: Claw Contraption](https://adventofcode.com/2024/day/13)

The problem is encoding a linear system in two variables. Namely, if Button A is $X+a_x$, $Y+a_y$, Button B is $X+b_x$, $Y+b_y$ and the prizes are $X=x$ and $Y=y$, then
pressing $u$ times button A and $v$ times button B is a solution if and only if:

$$
M
\left [
    \begin{matrix}
        u \\ 
        v
    \end{matrix}
\right ]
=
\left [
    \begin{matrix}
        x \\ 
        y
    \end{matrix}
\right ]
$$

where

$$
M = \left [
    \begin{matrix}
        a_x & b_x \\
        a_y & b_y
    \end{matrix}
\right ]
$$.

If $M$ is invertible, then there is a unique solution $M^{-1} \left[ \begin{matrix} x & y \end{matrix} \right]^T$.

If $M$ is not invertible and the rank of the nullspace is 1, then there are either no solutions or infinitely many solutions. The problem reduces itself to solving an integer linear programming problem: we have to minimize $3u+v$ subject to $au+bv=c$ with $u \geq 0$ and $v \geq 0$. Because the latter is a standard diophantine equation in two variables, it's easy to enumerate its solutions.

If $M$ is not invertible and the rank of the nullspace is 2 then $M = \mathbf{0}$ and the solution is trivial.

As is frequent for Advent of Code, the input is _less general than it could have been just by reading the problem description_. In fact, all matrices are invertible, making an implementation trivial.

In [15]:
def get_machines(raw):
    values = re.finditer(r'''Button A: X\+(\d+), Y\+(\d+)
Button B: X\+(\d+), Y\+(\d+)
Prize: X=(\d+), Y=(\d+)''', raw)
    for ax, ay, bx, by, x, y in map(lambda u: map(int, u.groups()), values):
        yield [[ax, bx], [ay, by]], [x, y]
       
class NoSolution(Exception): pass

def solve_linear_2x2(A, x):
    [[a, b], [c, d]] = A
    [u, v] = x
    det = a * d - b * c
    if all(x % det == 0 for x in {d*u - b*v, a*v - c*u}):
        return (
            (+d*u - b*v) // det,
            (-c*u + a*v) // det
        )
    else: raise NoSolution()

def tokens_for_machine(A, x, offset=0):
    try:
        u, v = solve_linear_2x2(A, [x[0] + offset, x[1] + offset])
        return 3 * u + v
    except NoSolution: return 0


A = sum(tokens_for_machine(A, x) for A, x in get_machines(aocin(13)))
assert A == 37686

A = sum(tokens_for_machine(A, x, offset=10**13) for A, x in get_machines(aocin(13)))
assert A == 77204516023437

## [Day 14: Restroom Redoubt](https://adventofcode.com/2024/day/14)

We assume that a Christmas Tree is "highly symmetric along the $y$ axis".

In [16]:
def get_robots(raw):
    for m in re.finditer(r'p=(-?\d+),(-?\d+) v=(-?\d+),(-?\d+)', raw):
        yield tuple(map(int, m.groups()))

def find_factor(robots, t, bounds):
    quadrant = Counter()
    bx, by = bounds
    for px, py, vx, vy in robots:
        fx = (px + vx * t) % bx
        fy = (py + vy * t) % by
        if fx > bx // 2 and fy > by // 2: quadrant.update('1')
        if fx > bx // 2 and fy < by // 2: quadrant.update('2')
        if fx < bx // 2 and fy > by // 2: quadrant.update('3')
        if fx < bx // 2 and fy < by // 2: quadrant.update('4')
    return prod(quadrant.values())

class RobotGrid:
    def __init__(self, robots, bounds):
        self.bx, self.by = bounds
        self.robots = dict(enumerate(robots))
        self.pos = {k: (v[0], v[1]) for k, v in self.robots.items()}

    def step(self):
        for n, (_, _, vx, vy) in self.robots.items():
            x, y = self.pos[n]
            self.pos[n] = (x + vx) % self.bx, (y + vy) % self.by

    def symmetry_score(self):
        S = set(self.pos.values())
        T = {(self.bx - x - 1, y) for x, y in S}
        return len(S.symmetric_difference(T))

    def print_grid(self):
        return '\n'.join(
            ''.join(
                'X' if (j, i) in self.pos.values() else '.'
                for j in range(self.bx)
            )
            for i in range(self.by)
        )

def find_christmas_tree(robots, bounds, max_steps=10_000):
    min_so_far, min_iter, min_grid = inf, None, None
    grid = RobotGrid(robots, bounds)
    for i in range(1, max_steps):
        grid.step()
        if (z := grid.symmetry_score()) < min_so_far:
            min_so_far, min_iter, min_grid = z, i, grid.print_grid()
    return min_so_far, min_iter, min_grid


A = find_factor(get_robots(aocin(14)), 100, (101, 103))
assert A == 229632480


_, A, min_grid = find_christmas_tree(get_robots(aocin(14)), (101, 103))
assert A == 7051

This is the resulting picture:

In [17]:
print('\n'.join(
    s[35:-35]
    for s in min_grid.split('\n')[30:63]
))

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X.............................X
X.............................X
X.............................X
X.............................X
X..............X..............X
X.............XXX.............X
X............XXXXX............X
X...........XXXXXXX...........X
X..........XXXXXXXXX..........X
X............XXXXX............X
X...........XXXXXXX...........X
X..........XXXXXXXXX..........X
X.........XXXXXXXXXXX.........X
X........XXXXXXXXXXXXX........X
X..........XXXXXXXXX..........X
X.........XXXXXXXXXXX.........X
X........XXXXXXXXXXXXX........X
X.......XXXXXXXXXXXXXXX.......X
X......XXXXXXXXXXXXXXXXX......X
X........XXXXXXXXXXXXX........X
X.......XXXXXXXXXXXXXXX.......X
X......XXXXXXXXXXXXXXXXX......X
X.....XXXXXXXXXXXXXXXXXXX.....X
X....XXXXXXXXXXXXXXXXXXXXX....X
X.............XXX.............X
X.............XXX.............X
X.............XXX.............X
X.............................X
X.............................X
X.............................X
X.......

## [Day 15: Warehouse Woes](https://adventofcode.com/2024/day/15)

What a pointless exercise.

In [18]:
class WarehouseGrid(Grid):
    def __init__(self, raw, expand=False):
        if expand:
            raw = ''.join(
                 '..' if c == '.'
            else '@.' if c == '@'
            else '##' if c == '#'
            else '[]' if c == 'O'
            else c
            for c in raw
        )
        super().__init__(raw)
        for i, j in product(range(self.height), range(self.width)):
            if self[i, j] == '@': 
                self.position = i, j
                self[i, j] = '.'
                break
        self.box = '[' if expand else 'O'

    def make_move(self, direction):
        match direction:
            case '^': dx, dy = -1,  0
            case 'v': dx, dy =  1,  0
            case '<': dx, dy =  0, -1
            case '>': dx, dy =  0,  1
        px, py = self.position
        if self.can_move(px, py, dx, dy):
            self.move(px, py, dx, dy)
            self.position = px+dx, py+dy

    def can_move(self, px, py, dx, dy):
        match self[px+dx, py+dy], dx:
            case '.', _: return True
            case '#', _: return False
            case ('[' | ']', 0) | ('O', _): return self.can_move(px+dx, py+dy, dx, dy)
            case '[', _: 
                return self.can_move(px+dx, py+dy, dx, dy) and self.can_move(px+dx, py+dy+1, dx, dy)
            case ']', _:
                return self.can_move(px+dx, py+dy, dx, dy) and self.can_move(px+dx, py+dy-1, dx, dy)

    def move(self, px, py, dx, dy):
        match self[px+dx, py+dy], dx:
            case ('[' | ']', 0) | ('O', _):
                self.move(px+dx, py+dy, dx, dy)
            case '[', _:
                self.move(px+dx, py+dy, dx, dy)
                self.move(px+dx, py+dy+1, dx, dy)
            case ']', _:
                self.move(px+dx, py+dy, dx, dy)
                self.move(px+dx, py+dy-1, dx, dy)
        self[px, py], self[px+dx, py+dy] = '.', self[px, py]
        
    def sum_coord(self):
        return sum(
            100*i + j 
            for i, j in product(range(self.height), range(self.width))
            if self[i, j] == self.box
        )

def get_sum_coord(raw_grid, moves, expand=False):
    grid = WarehouseGrid(raw_grid, expand=expand)
    for move in moves:
        grid.make_move(move)
    return grid.sum_coord()


raw_grid, raw_moves = aocin(15).split('\n\n')
A = get_sum_coord(raw_grid, raw_moves.replace('\n', ''))
assert A == 1349898

A = get_sum_coord(raw_grid, raw_moves.replace('\n', ''), expand=True)
assert A == 1376686

## [Day 16: Reindeer Maze](https://adventofcode.com/2024/day/16)

In [19]:
class MazeGrid(Grid):
    def __init__(self, raw):
        super().__init__(raw)
        for i, j in product(range(self.height), range(self.width)):
            if self[i, j] == 'S': self.start = i, j
            if self[i, j] == 'E': self.end = i, j


def get_min_path(grid: MazeGrid):
    H = [(0, (*grid.start, 0, 1), None)]
    D = {}
    while H:
        dist, (i, j, hi, hj), prev = heappop(H)
        if grid[i, j] == '#': continue
        try:
            opt_dist, opt_prev = D[i, j, hi, hj]
            if opt_dist == dist: opt_prev.append(prev)
            continue
        except KeyError: pass
        D[i, j, hi, hj] = dist, [prev]
        heappush(H, (dist + 1, (i+hi, j+hj, hi, hj), (i, j, hi, hj)))
        heappush(H, (dist + 1000, (i, j, hj, hi), (i, j, hi, hj)))
        heappush(H, (dist + 1000, (i, j, -hj, -hi), (i, j, hi, hj)))

    directions = ((0, 1), (0, -1), (1, 0), (-1, 0))
    min_dist = min(D[*grid.end, *h][0] for h in directions)

    visited = set()
    for h in directions:
        if D[*grid.end, *h][0] != min_dist: continue
        S = [(*grid.end, *h)]
        while S:
            if (u := S.pop()):
                i, j, di, dj = u
                visited.add((i, j))
                S.extend(D[i, j, di, dj][1])

    return min_dist, visited

grid = MazeGrid(aocin(16))
A, S = get_min_path(grid)
assert A == 93436

A = len(S)
assert A == 486

## [Day 17: Chronospatial Computer](https://adventofcode.com/2024/day/17)

Let's manually analyze the code.

    Intcode     Mnemonic
    2, 4        b = a % 8
    1, 1        b = b ^ 1
    7, 5        c = a >> b
    4, 7        b = b ^ c
    1, 4        b = b ^ 4
    0, 3        a = a // 8
    5, 5        print (b % 8)
    3, 0        jump to the first instruction if a != 0

We can simply translate this directly to Python code as a While loop.



In [20]:
def exec(n):
    while True:
        b = (n % 8) ^ 1
        c = n >> b
        yield (b ^ c ^ 4) % 8
        n = n // 8
        if n == 0: break


register = int(re.search(r'Register A: (\d+)', aocin(17)).group(1))
A = ','.join(map(str, exec(register)))
assert A == '1,3,7,4,6,4,2,3,5'

For part 2, we may observe that the program is effectively scanning the digits of the input in base 8, right to left:

    2, 4        b = a % 8
    ...
    0, 3        a = a // 8
    ...

...and the output only depends on the current digit and digits positioned to its left:

    7, 5        c = a >> b

This means that the final digit of the output depends only on the leftmost base 8 digit of the input!
A simple recursive algorithm can therefore find all inputs that produce the desired output.

In [21]:
def discover_input(program, prev=0):
    if not program: yield prev; return
    for i in range(8):
        if next(exec(8*prev+i)) == program[-1]:
            yield from discover_input(program[:-1], 8*prev+i)


program = [*map(int, re.search(r'Program: ((?:\d,)+\d)', aocin(17)).group(1).split(','))]
A = min(discover_input(program))
assert A == 202367025818154

Interestingly, the computational model described by the problem is _not_ Turing-complete. The only arithmetic operations performed are Euclidean division,
modulo and exclusive OR, all of which have a result that cannot be greater than their arguments. The number of states is therefore bounded by $N^3 \times L$, where $N$ is the greatest inital register value, and $L$ is the length of the program.

## [Day 18: RAM Run](https://adventofcode.com/2024/day/18)

In [22]:
def min_path(fallen, start=(0, 0), end=(70, 70), height=71, width=71):
    can_walk = lambda i, j: (
            0 <= i < height
        and 0 <= j < width
        and (i, j) not in fallen
    )
    H, D = [(0, start, None)], {}
    while H:
        dist, (i, j), prev = heappop(H)
        if not can_walk(i, j): continue
        if (i, j) in D: continue
        D[i, j] = prev
        if (i, j) == end: break
        for ii, jj in ((i+1, j), (i-1, j), (i, j-1), (i, j+1)):
            heappush(H, (1+dist, (ii, jj), (i, j)))
    else: return None

    path = set()
    i, j = end
    while (u := D[i, j]):
        path.add((i, j))
        i, j = u
    return path


def first_cut(fallen, start=(0, 0), end=(70, 70), height=71, width=71):
    out = set()
    f = lambda: min_path(out, start, end, height, width)
    current_path = f()
    for i, j in fallen:
        out.add((i, j))
        if (i, j) in current_path and not (current_path := f()):
            return i, j


fallen = list(map(lambda u: tuple(map(int, u.split(','))), aocin(18).split('\n')))
A = len(min_path(set(fallen[:1024])))
assert A == 316

A = first_cut(fallen)
assert A == (45, 18)

## [Day 19: Linen Layout](https://adventofcode.com/2024/day/19)

A possible optimization is to construct a trie from the input, but it wasn't necessary.

In [23]:
@cache
def n_compositions(word, pcs):
    return 1 if not word else sum(
        n_compositions(word[len(p):], pcs) for p in pcs
        if word.startswith(p)
    )


raw_pcs, raw_words = aocin(19).split('\n\n')
words = raw_words.split('\n')
pcs = frozenset(u.strip() for u in raw_pcs.split(','))

A = sum(bool(n_compositions(w, pcs)) for w in words)
assert A == 258

A = sum(n_compositions(w, pcs) for w in words)
assert A == 632423618484345

## [Day 20: Race Condition](https://adventofcode.com/2024/day/20)

In [24]:
class RaceConditionGrid(Grid):
    def __init__(self, raw):
        super().__init__(raw)
        for i, j in product(range(self.height), range(self.width)):
            if self[i, j] == 'S': self.start = i, j
            if self[i, j] == 'E': self.end = i, j

def single_source_shortest_paths(grid):
    Q, D = [(0, *grid.end)], {}
    while Q:
        dist, i, j = Q.pop()
        if (i, j) in D: continue
        if grid[i, j] == '#': continue
        D[i, j] = dist
        for ii, jj in ((i+1, j), (i-1, j), (i, j+1), (i, j-1)):
            Q.append((dist+1, ii, jj))
    return D

def possible_cheats(D, cheat_len=2, saving=100):
    def landing_pads(i, j):
        for di in range(-cheat_len, cheat_len+1):
            dr = cheat_len - abs(di)
            for dj in range(-dr, dr+1):
                yield i+di, j+dj
    for i, j in D:
        for ei, ej in landing_pads(i, j):
            if (z := D[i, j] - (D.get((ei, ej), inf) + abs(i-ei) + abs(j-ej))) >= saving:
                yield (i, j), (ei, ej), z


grid = RaceConditionGrid(aocin(20))
D = single_source_shortest_paths(grid)
A = len(set(possible_cheats(D, cheat_len=2, saving=100)))
assert A == 1399

A = len(set(possible_cheats(D, cheat_len=20, saving=100)))
assert A == 994807

## [Day 21: Keypad Conundrum](https://adventofcode.com/2024/day/21)

In [35]:
from itertools import pairwise
from dataclasses import dataclass

numeric_geometry = ('789', '456', '123', ' 0A')
directional_geometry = (' ^A', '<v>')

@dataclass(frozen=True, slots=True)
class Concat:
    x: tuple

@dataclass(frozen=True, slots=True)
class Choice:
    x: tuple

@cache
def press(geometry, o, d):
    def locate(c):
        for n, i in enumerate(geometry):
            if (z := i.find(c)) != -1: return n, z
    ox, oy = locate(o)
    dx, dy = locate(d)
    void = locate(' ')
    P, M = [('', ox, oy)], set()
    while P:
        m, ox, oy = P.pop()
        if (ox, oy) == (dx, dy): M.add(m); continue
        if ox < dx and (ox+1, oy) != void: P.append((m+'v', ox+1, oy))
        if ox > dx and (ox-1, oy) != void: P.append((m+'^', ox-1, oy))
        if oy < dy and (ox, oy+1) != void: P.append((m+'>', ox, oy+1))
        if oy > dy and (ox, oy-1) != void: P.append((m+'<', ox, oy-1))
    return M

@cache
def manipulate(path, geometry):
    return Concat(tuple(
        Choice(tuple(h + 'A' for h in press(geometry, p, c)))
        for p, c in pairwise('A' + path)
    ))

@cache
def manipulate_chained(path):
    match path:
        case str():     return manipulate(path, directional_geometry)
        case Choice(x): return Choice(tuple(map(manipulate_chained, x)))
        case Concat(x): return Concat(tuple(map(manipulate_chained, x)))

def code_complexity(code, depth):
    @cache
    def min_path(t):
        match t:
            case str():     return len(t)
            case Choice(x): return min(map(min_path, x))
            case Concat(x): return sum(map(min_path, x))

    @cache
    def prune(t):
        match t:
            case str():
                return t
            case Choice(x):
                m = min(map(min_path, x))
                return Choice(tuple(
                    prune(t) for t in x
                    if m == min_path(t)
                ))
            case Concat(x):
                return Concat(tuple(map(prune, x)))


    start = manipulate(code, numeric_geometry)
    for i in range(depth):
        start = manipulate_chained(start)
        start = prune(start)
    return int(code[:3]) * min_path(start)


codes = aocin(21).split('\n')
A = sum(code_complexity(x, 2) for x in codes)
assert A == 188398

# A = sum(code_complexity(x, 25) for x in codes)
# A

## [Day 22: Money Market](https://adventofcode.com/2024/day/22)

In [144]:
def seed_prng(n):
    yield n
    while True:
        n ^= ((n << 6) % (2 ** 24))
        n ^= ((n >> 5) % (2 ** 24))
        n ^= ((n << 11) % (2 ** 24))
        yield n

def best_return(numbers):
    seq = [list(islice(seed_prng(n), 2000)) for n in numbers]
    scores = Counter()
    for s in seq:
        part = Counter()
        for i in range(0, len(s)-5):
            dx = tuple((b%10-a%10) for a, b in pairwise(s[i:i+5]))
            if dx not in part:
                part[dx] = s[i+4] % 10
        scores += part
    return scores


numbers = list(map(int, aocin(22).split('\n')))
A = sum(next(islice(seed_prng(n), 2000, None)) for n in numbers)
assert A == 13764677935

A = max(best_return(numbers).values())
assert A == 1619